
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Getting Started with MLflow and Mosaic AI Model Serving

In this lesson, we will take a second look at MLflow outside the perspective of AutoML. To this end, this notebook uses a random forest classifier as an example model so we can demonstrate how to log a model. We will demonstrate how to use Mosaic AI Model Serving for real-time inferencing. Finally, we give a brief overview of how to utilize Workflows for machine learning. 

**Learning Objectives:**

_By the end of this demo, you will be able to:_


1. Understand the basics of how to track and manage models with MLflow:
    - Train, track, and register a regression model. 
    - Manage model staging programmatically and using the UI.


2. Understand how to serve your model with Mosaic AI Model Serving
    - Use the UI to serve your model.
    - Query the endpoint using the UI.
    - Explore the metrics and logs using the built-in dashboard. 

3. Introduction to ML With Workflows:
    - Use the UI to demonstrate notebook automation for ML tasks via Databricks Workflows.


## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-cpu-ml-scala2.12 15.4.x-scala2.12**


## Classroom Setup

To get into the lesson, we first need to build some data assets and define some configuration variables required for this demonstration. When running the following cell, the output is hidden so our space isn't cluttered. To view the details of the output, you can hover over the next cell and click the eye icon. 

The cell after the setup, titled `View Setup Variables`, displays the various variables that were created. 

The classroom setup will create the following assets:  
- Catalog (see below for naming)
- Database  (see below for naming)
- Table `customer_details`
- Table `customer_features`
- Model (see below for naming)
- Model Serving Endpoint **get-started-model-serving-endpoint**

This Model Serving Endpoint will be available to everyone and deployed once. Once it is created, you will see the message "Endpoint 'get-started-model-serving-endpoint' already exists." when trying to create it again. This is to limit resource deployment within this workspace. The first creation will take between 5 and 10 minutes. **Do not delete or modify this endpoint in any way.**

In [0]:
%run ../Includes/Classroom-Setup-03

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| dropping the catalog "labuser8027617_1732916382_2w2l_da"...(1 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/get-started-with-databricks-for-machine-learning/v01"

Validating the locally installed datasets:
| listing local files...(0 seconds)
| validation completed...(0 seconds total)
Creating & using the catalog "labuser8027617_1732916382_2w2l_da"...(2 seconds)

Predefined tables in "labuser8027617_1732916382_2w2l_da.default":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/labuser8027617_1732916382@vocareum.com/get-started-with-databricks-for-machine-learning
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/get-started-with-databricks-for-machine-learning/v01

Setup completed (8 seconds)


2024/11/29 21:53:19 INFO databricks.ml_features._compute_client._compute_client: Setting columns ['customerid'] of table 'labuser8027617_1732916382_2w2l_da.default.customer_features' to NOT NULL.
2024/11/29 21:53:24 INFO databricks.ml_features._compute_client._compute_client: Setting Primary Keys constraint ['customerid'] on table 'labuser8027617_1732916382_2w2l_da.default.customer_features'.
2024/11/29 21:53:25 INFO databricks.ml_features._compute_client._compute_client: Created feature table 'labuser8027617_1732916382_2w2l_da.default.customer_features'.
2024/11/29 21:53:48 INFO mlflow.tracking.fluent: Experiment with name '/Users/labuser8027617_1732916382@vocareum.com/get-started-ml-experiment' does not exist. Creating a new experiment.
/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

2024/11/29 21:54:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run end_to_end_ml_on_databricks_run at: dbc-40a12f74-fe09.cloud.databricks.com/ml/experiments/107295324166451/runs/f7de8f6aba3648a49f96c5de3a19b742.
2024/11/29 21:54:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: dbc-40a12f74-fe09.cloud.databricks.com/ml/experiments/107295324166451.
Successfully registered model 'labuser8027617_1732916382_2w2l_da.default.my_model_labuser8027617-1732916382-2w2l-da-gsml'.


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Created version '1' of model 'labuser8027617_1732916382_2w2l_da.default.my_model_labuser8027617-1732916382-2w2l-da-gsml'.


Endpoint 'get-started-model-serving-endpoint' already exists.
Failed to update permissions: {"error_code":"PERMISSION_DENIED","message":"labuser8027617_1732916382@vocareum.com does not have Manage permissions on Inference Endpoint with ID: 90c7e88605714e35b03daa6176d2d3c4. Please contact the owner or an administrator for access."}


In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"User DB Location:  {DA.paths.user_db}")

Username:          labuser8027617_1732916382@vocareum.com
Catalog Name:      labuser8027617_1732916382_2w2l_da
Schema Name:       default
Working Directory: dbfs:/mnt/dbacademy-users/labuser8027617_1732916382@vocareum.com/get-started-with-databricks-for-machine-learning
User DB Location:  None


In [0]:
%sql
SHOW TABLES

database,tableName,isTemporary
default,customer_details,false
default,customer_features,false


## Part 1: Data Preparation for MLflow

For this first part, we will read in a customer table, create a feature table, and register that feature table to Databricks Feature Store. This is all performed in the background and was covered in `01 - EDA and Feature Engineering`. Next, we will separate out our features from the target variable and perform a train-test split. 

Note: We are working with a Pandas DataFrame called `training_df`.

In [0]:
# Register feature table to Feature Store and load it
training_df = DA.create_feature_store()

In [0]:

# Use the training dataset to store variables X, the features, and y, the target variable. 
X = training_df.drop("Churn", axis=1)
y = training_df["Churn"]

# Convert categorical labels to numerical labels
y = y.map({'Yes': 1.0, 'No': 0.0})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Part 2: Training And Model Tracking/Management With MLflow

Now that we have our training set ready to go, the next step is to train a model using the `sklearn` library. We will build a random forest classification model, tracking the F1-score for a single run. We will initiate the tracking before creating the model using `mlflow.start_run()` as the context manager. Within this manager we will:

1. Initialize the random forest classifier
2. Fit the model
Make a prediction using our test set
3. Log the F1-score metric as `test_f1`
4. Capture the artifacts for model tracking and management using the flavor `mlflow.sklearn`. *Flavor* in this context simply means that MLflow will package our scikit-learn model in a consistent and standardized way. If we wished to use a different ML library, we would use a different *flavor*.

Finally, we will register the model to Unity Catalog. Note, Databricks does not recommend registering your model at the Workspace level. Recall that we did this using the UI in the previous lab `02 - Experimentation with Mosaic AI AutoML`.



In [0]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# set the path for mlflow experiment
mlflow.set_experiment(f"/Users/{DA.username}/M05-End-to-end-ML-Lakehouse")

with mlflow.start_run(run_name = 'end_to_end_ml_on_databricks_run') as run:  
    # Initialize the Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

    # Fit the model on the training data
    rf_classifier.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = rf_classifier.predict(X_test)

    # Enable automatic logging of input samples, metrics, parameters, and models
    mlflow.sklearn.autolog(
        log_input_examples = True,
        silent = True
    )

    mlflow.log_metric("test_f1", f1_score(y_test, y_pred))
        
    mlflow.sklearn.log_model(
        rf_classifier,
        artifact_path = "model-artifacts", 
        input_example=X_train[:3],
        signature=infer_signature(X_train, y_train)
    )

    model_uri = f"runs:/{run.info.run_id}/model-artifacts"

2024/11/29 21:58:30 INFO mlflow.tracking.fluent: Experiment with name '/Users/labuser8027617_1732916382@vocareum.com/M05-End-to-end-ML-Lakehouse' does not exist. Creating a new experiment.


Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

2024/11/29 21:58:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run end_to_end_ml_on_databricks_run at: dbc-40a12f74-fe09.cloud.databricks.com/ml/experiments/107295324166452/runs/45d3d901815848fa99c3dc519105be65.
2024/11/29 21:58:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: dbc-40a12f74-fe09.cloud.databricks.com/ml/experiments/107295324166452.


In [0]:
# Modify the registry uri to point to Unity Catalog
mlflow.set_registry_uri("databricks-uc")

# Define the model name 
model_name = f"{DA.catalog_name}.{DA.schema_name}.my_model_{DA.unique_name('-')}"

# Register the model in the model registry
registered_model = mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'labuser8027617_1732916382_2w2l_da.default.my_model_labuser8027617-1732916382-2w2l-da-gsml' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Created version '2' of model 'labuser8027617_1732916382_2w2l_da.default.my_model_labuser8027617-1732916382-2w2l-da-gsml'.


Notice that you will now have an additional version under your model. Navigate to your model in Catalog explorer. You will find version 1 (created during the classroom setup with alias **staging**) and version 2, which you must created.

In [0]:
from mlflow.tracking.client import MlflowClient

# Initialize an MLflow Client
client = MlflowClient()

# Assign a "staging" alias to model version 1
client.set_registered_model_alias(
    name= registered_model.name,  # The registered model name
    alias="dev",  # The alias representing the dev environment
    version=registered_model.version  # The version of the model you want to move to "dev"
)

**Note🚨 : Below instructions are only for demonstration purposes. Please do not provision an endpoint. It is already created during the workspace setup.**


## Part 3: Mosaic AI Model Serving

**Setting Up Model Serving**

We can create Model Serving endpoints with the Databricks Machine Learning API or the Databricks Machine Learning UI. An endpoint can serve any registered Python MLflow model in the **Model Registry**.

In order to keep it simple, in this demo, we are going to use the Model Serving UI for creating, managing and using the Model Serving endpoints. We can create model serving endpoints with the **"Serving"** page UI or directly from registered **"Models"** page.  

Let's go through the steps of creating a model serving endpoint in Models page. **You will not actually create the endpoint.**

- Go to **Models**. 

- Select **Unity Catalog** at the top and select **Owned by me** as well.

- Select the model you want to serve under the **Name** column. Notice this will take you to the Catalog menu. 

- Click the **Serve this model** button on the top right. This will take you to the **Serving endpoints** screen.

- Next in **General**, enter in a name. This name should be unique like your first and last name. For example, **get-started-model-serving-endpoint**.

- There are several configurations under **Served entities** that we will not discuss here. Leave **Entity** and **Compute type** to default values. For **Compute scale-out**, select **small**. You can select **Scale to zero** for this lesson as well. We will be deleting the endpoint at the end of this lesson, so this doesn't matter too much for our purposes. 

- **Do not click Create** at the bottom right. The above instructions are only for demonstration purposes. **Do not provision an endpoint.**

    - If you happen to accidentally create an endpoint, you can navigate to the left side bar and click on **Serving**. Then click on the model you began provisioning and click on the 3 vertical dots at the top right. Select **Delete**. Again, **Do not provision an endpoint.**

- If you do click create, you will be met with an error saying "Endpoint with name 'get-started-model-serving-endpoint' already exists." This is because we already setup this endpoint during the setup to this course. **Do not provision an endpoint by changing the name.**



### Create Model Serving Endpoint with Python

**Note**:  We will not be creating a new endpoint in this **demonstration**. However, the code implementation is shown below.

```

from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")
endpoint_name = f"{DA.username}-model"
endpoint_name = endpoint_name.replace("@databricks.com", "").replace('.', '-')

# Check if the endpoint already exists
try:
    # Attempt to get the endpoint
    existing_endpoint = client.get_endpoint(endpoint_name)
    print(f"Endpoint '{endpoint_name}' already exists.")
except Exception as e:
    # If not found, create the endpoint
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        print(f"Creating a new endpoint: {endpoint_name}")
        endpoint = client.create_endpoint(
            name=endpoint_name,
            config={
                "served_entities": [
                    {
                        "name": "my-model",
                        "entity_name": model_name,
                        "entity_version": "1",
                        "workload_size": "Small",
                        "scale_to_zero_enabled": True
                    }
                ],
                "traffic_config": {
                    "routes": [
                        {
                            "served_model_name": "my-model",
                            "traffic_percentage": 100
                        }
                    ]
                }
            }
        )
    else:
        print(f"An error occurred: {e}")
```

### Query Serving Endpoint

Let's use the deployed model for real-time inference. Here’s a step-by-step guide for querying an endpoint in Databricks Model Serving:

- Go to the **Serving** endpoints page and select the endpoint you want to query.

- Click **Use** button the top right corner.

- There are 4 methods for querying an endpoint; **browser**, **CURL**, **Python**, and **SQL**. For now, let's use the easiest method; querying right in the **browser** window. In this method, we need to provide the input parameters in JSON format. Since we used `mlflow.sklearn.autolog()` with `log_input_examples = True`, we registered an example with MLflow, which appear automatically when selecting **browser**.

- Click **Send request**.

- **Response** field on the right panel will show the result of the inference.

## Part 4: ML With Workflows

### Creating a Job

Now, let's create a workflow job to run this notebook:

1. Start by navigating to **Workflows** on the left sidebar.
   
2. Click **Create job** at the top right.

3. In the presented menu, enter a name under **Task Name**. For example, you can use **test_task_name**.

4. Under **Type**, ensure that **Notebook** is selected. Also, make sure **Workspace** is selected under **Source**. Both should be selected by default.

5. Next, we will attach this notebook to the job. Go to **Path**. Click the drop-down menu. You will be presented with a menu that will allow you to navigate to this notebook. By default, you will already be in your home workspace. It will look like `labuserXXXXXXX@vocareum.com`. Click the source folder (there will only be a single option listed other than **Trash**). Select `M02 - Databricks for Machine Learning` and then select `03 - Getting Started with MLflow and Mosaic AI Model Serving`. Click **Confirm** at the bottom right of the **Select Notebook** menu. 

6. Under **Compute**, you'll see **Serverless** is automatically selected.
Use the drop-down menu to select your existing All-Purpose **cluster** (DBR 15.4 LTS ML).

7. You’ll see additional options, such as **dependent libraries**, **parameters**, **notifications**, **retries**, and **duration threshold**. You can leave these options unpopulated for this lesson. 

8. Click **Create Task**. A message will appear at the top right indicating your task was successfully created.

9. In the right menu, there are various options. For example, you can add tags, set job parameters, and configure schedules for runs. We'll leave these alone for this lesson.

10. Finally, to give the job a name, double-click the title at the top left and enter an appropriate name. For example, `Workflow1`.

11. When you are ready to run the job, click **Run Now** at the top right. A note will appear at the top right indicating a run has been initiated. Note that due to the clean-up in the next cell, your data assets will be missing when inspecting your catalog after the job finishes. 
    - You can also run this workflow from the parent menu (navigate to **Workflows** on the sidebar within Databricks) by clicking the **Run now** button ▶. 

### Inspect Your Run

After your job completes its run, you can inspect it:

1. Select **Workflows** again on the sidebar menu. 

2. Click on the name of the job you just created. 

3. There are a few ways to view the notebook you just ran. You can do any of the following and it will take you to the same location:
    - Click on **go to the latest successful run**.
    - Click on the date under **Start time**.
    - Click on the green bar (meaning a successful run) displayed within the diagram. If the bar is red, that means your job failed, but you can still inspect the notebook that ran. This option will also display the workflow configuration you setup previously. 

4. This will take you to a static copy of the notebook. You cannot edit it, but you can view the outputs of each cell. Confirm that all cells have run successfully. 

Of course, this is a simple example of creating a job with a single notebook. In practice, we use the various options mentioned earlier to tie together complex pipelines.

# Classroom Clean-up

After completing the demo, it's important to clean up any resources that were created.

Run the following cell to remove lessons-specific assets created during this lesson.

In [0]:
DA.cleanup()

Resetting the learning environment:
| dropping the catalog "labuser8027617_1732916382_2w2l_da"...(0 seconds)

Validating the locally installed datasets:
| listing local files...(0 seconds)
| validation completed...(0 seconds total)


# Conclusions And Next Steps

In this demo, we covered how to track, manage, and stage models with MLflow, as well as serve models using Mosaic AI Model Serving. You learned to automate model deployment workflows by parameterizing variables in the UI, streamlining the pipeline from model training to inference.


&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>